<a href="https://colab.research.google.com/github/AhoodAlsuhaibani/TeacherAssistantChatbot/blob/main/Teacher's_Assistant_Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q torch transformers transformers accelerate bitsandbytes langchain sentence-transformers faiss-gpu openpyxl pacmap

In [ ]:

!pip install -U langchain-community

  Using cached langchain_community-0.2.1-py3-none-any.whl (2.1 MB)
  Using cached langchain-0.2.1-py3-none-any.whl (973 kB)
  Using cached langchain_core-0.2.1-py3-none-any.whl (308 kB)
  Using cached langchain_text_splitters-0.2.0-py3-none-any.whl (23 kB)
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.1.52
    Uninstalling langchain-core-0.1.52:
      Successfully uninstalled langchain-core-0.1.52
  Attempting uninstall: langchain-text-splitters
    Found existing installation: langchain-text-splitters 0.0.2
    Uninstalling langchain-text-splitters-0.0.2:
      Successfully uninstalled langchain-text-splitters-0.0.2
  Attempting uninstall: langchain
    Found existing installation: langchain 0.1.20
    Uninstalling langchain-0.1.20:
      Successfully uninstalled langchain-0.1.20
  Attempting uninstall: langchain-community
    Found existing installation: langchain-community 0.0.38
    Uninstalling langchain-community-0.0.38:
      Successful

In [ ]:
import os
import sys

In [ ]:
!pip install langchain_community

In [ ]:
from langchain.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores.utils import DistanceStrategy


EMBEDDING_MODEL_NAME = "thenlper/gte-small"
embedding_model = HuggingFaceEmbeddings(
    model_name=EMBEDDING_MODEL_NAME,
    multi_process=True,
    model_kwargs={"device": "cuda"},
    encode_kwargs={"normalize_embeddings": True},
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install pypdf

In [ ]:
from langchain_community.document_loaders import PyPDFLoader

In [ ]:
code_docs=[]
from langchain_community.document_loaders import NotebookLoader
for file in os.listdir("/content/drive/MyDrive/CODES"):
  if file.endswith(".ipynb"):
    ipynb_path="/content/drive/MyDrive/CODES/"+file
    loader = NotebookLoader(
    ipynb_path,
    include_outputs=True,
    max_output_length=20,
    remove_newline=True,
)
    code_docs.extend(loader.load())

In [ ]:
pdf_docs=[]
for file in os.listdir("/content/drive/MyDrive/PDFS "):
  if file.endswith(".pdf"):
    pdf_path="/content/drive/MyDrive/PDFS /"+file
    loader=PyPDFLoader(pdf_path)
    pdf_docs.extend(loader.load())

In [ ]:
from langchain_community.document_loaders import YoutubeLoader
%pip install --upgrade --quiet  youtube-transcript-api

vid_docs=[]
vid_list=["https://www.youtube.com/watch?v=HXV3zeQKqGY","https://www.youtube.com/watch?v=_uQrJ0TkZlc","https://www.youtube.com/watch?v=JonnQuQDThc","https://www.youtube.com/watch?v=d2kxUVwWWwU","https://www.youtube.com/watch?v=IA3WxTTPXqQ","https://www.youtube.com/watch?v=igKTO7lQxNo","https://www.youtube.com/watch?v=r-uOLxNrNk8"]

for vid in vid_list:
  loader=YoutubeLoader.from_youtube_url(
    vid, add_video_info=False
)
  vid_docs.extend(loader.load())

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter


NOTEBOOK_SEPARATORS = [
    "\n# ",        # Headings
    "\n## ",       # Subheadings
    "\n### ",      # Sub-subheadings
    "\n---\n",     # Horizontal rule
    "\n\n",        # Double newline
    "\n",          # Single newline
    " ",           # Space
    "",            # Empty string
]


PDF_SEPARATORS = ["\f"]



In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from transformers import AutoTokenizer
from typing import Optional, List, Tuple
from langchain.docstore.document import Document as LangchainDocument



EMBEDDING_MODEL_NAME = "thenlper/gte-small"


def split_documents(
    chunk_size: int,
    knowledge_base: List[LangchainDocument],
    tokenizer_name: Optional[str] = EMBEDDING_MODEL_NAME,
) -> List[LangchainDocument]:
    """
    Split documents into chunks of maximum size `chunk_size` tokens and return a list of documents.
    """
    text_splitter = RecursiveCharacterTextSplitter.from_huggingface_tokenizer(
        AutoTokenizer.from_pretrained(tokenizer_name),
        chunk_size=chunk_size,
        chunk_overlap=int(chunk_size / 10),
        add_start_index=True,
        strip_whitespace=True,
        separators=PDF_SEPARATORS,
    )


    docs_processed = []
    for doc in knowledge_base:
        docs_processed += text_splitter.split_documents([doc])

    # Remove duplicates
    unique_texts = {}
    docs_processed_unique = []
    for doc in docs_processed:
        if doc.page_content not in unique_texts:
            unique_texts[doc.page_content] = True
            docs_processed_unique.append(doc)

    return docs_processed_unique


docs_processed_pdf = split_documents(
   512,
    pdf_docs,
    tokenizer_name=EMBEDDING_MODEL_NAME,
)


In [ ]:
def split_documents(
    chunk_size: int,
    knowledge_base: List[LangchainDocument],
    tokenizer_name: Optional[str] = EMBEDDING_MODEL_NAME,
) -> List[LangchainDocument]:
    """
    Split documents into chunks of maximum size `chunk_size` tokens and return a list of documents.
    """
    text_splitter = RecursiveCharacterTextSplitter.from_huggingface_tokenizer(
        AutoTokenizer.from_pretrained(tokenizer_name),
        chunk_size=chunk_size,
        chunk_overlap=int(chunk_size / 10),
        add_start_index=True,
        strip_whitespace=True,
    )


    docs_processed = []
    for doc in knowledge_base:
        docs_processed += text_splitter.split_documents([doc])

    # Remove duplicates
    unique_texts = {}
    docs_processed_unique = []
    for doc in docs_processed:
        if doc.page_content not in unique_texts:
            unique_texts[doc.page_content] = True
            docs_processed_unique.append(doc)

    return docs_processed_unique


docs_processed_vid = split_documents(
   512,
    vid_docs,
    tokenizer_name=EMBEDDING_MODEL_NAME,
)


In [ ]:
def split_documents(
    chunk_size: int,
    knowledge_base: List[LangchainDocument],
    tokenizer_name: Optional[str] = EMBEDDING_MODEL_NAME,
) -> List[LangchainDocument]:
    """
    Split documents into chunks of maximum size `chunk_size` tokens and return a list of documents.
    """
    text_splitter = RecursiveCharacterTextSplitter.from_huggingface_tokenizer(
        AutoTokenizer.from_pretrained(tokenizer_name),
        chunk_size=chunk_size,
        chunk_overlap=int(chunk_size / 10),
        add_start_index=True,
        strip_whitespace=True,
        separators=NOTEBOOK_SEPARATORS,
    )


    docs_processed = []
    for doc in knowledge_base:
        docs_processed += text_splitter.split_documents([doc])

    # Remove duplicates
    unique_texts = {}
    docs_processed_unique = []
    for doc in docs_processed:
        if doc.page_content not in unique_texts:
            unique_texts[doc.page_content] = True
            docs_processed_unique.append(doc)

    return docs_processed_unique


docs_processed_code = split_documents(
   512,
    code_docs,
    tokenizer_name=EMBEDDING_MODEL_NAME,
)


In [ ]:
print(docs_processed_pdf[0])
print(docs_processed_vid[0])
print(docs_processed_code[0])

page_content='1 - V 1.0/05032024  \nProgram Project  \nProgram Identification  \nProgram ID:  T5 \nProgram Title: Introduction to AI, Data Analytics & SQL  \n \nProgram Information:  \nProject Title:  Railway System Database Management System    \n \nDescription  \n \nThe railway management system project can help make the process of planning trips, booking tickets, reservations and last -minute cancellations more convenient. \nThe system will streamline the process for users, which will also help retain them as users. Create a database he dataset for this project will contain essential \ndetails, such as:    \na) Train information like ( Train number , speed, …etc  \nb) Station information like ( Station code , Station name , …etc)  \nc) Schedule information like (Trip code, departure city, arrival city, Departure time , Distance , Price    …etc)  \nd) Traveler information like (Name, phone number, age, ….etc)  \ne) Ticket information like (Date, trip number, client number, …etc)  \n 

In [ ]:
VECTORDB = FAISS.from_documents(
    docs_processed_pdf + docs_processed_code + docs_processed_vid, embedding_model,distance_strategy=DistanceStrategy.COSINE
)

In [ ]:
VECTORDB

In [ ]:
FAISS.save_local(VECTORDB, "faiss_index")

In [ ]:
VECTORDB = FAISS.load_local("faiss_index", embedding_model, allow_dangerous_deserialization=True)


In [ ]:
user_query = "what is NLP?"
print(f"\nStarting retrieval for {user_query=}...")
retrieved_docs = VECTORDB.similarity_search(query=user_query, k=5)
print(
    "\n==================================Top document=================================="
)
print(retrieved_docs[0].page_content)
print("==================================Metadata==================================")
print(retrieved_docs[0].metadata)


Starting retrieval for user_query='what is NLP?'...

==================================Top document==================================
for people who are not aware about nlp i would like to give a quick introduction about what nlp is and why is it important right so nlp is the abbreviation for natural language processing and nlp is a combination of using human language with a bit of programming and artificial intelligence so that we are able to convert text into a format that can be you know run through algorithms and used for prediction right so we are converting the unstructured text data into structured data and then gaining insights this entire process is called nlp now you might have
==================================Metadata==================================
{'source': 'igKTO7lQxNo', 'start_index': 352664}


In [ ]:
from transformers import pipeline
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

READER_MODEL_NAME = "HuggingFaceH4/zephyr-7b-alpha"  #"HuggingFaceH4/zephyr-7b-beta"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)
model = AutoModelForCausalLM.from_pretrained(
    READER_MODEL_NAME, quantization_config=bnb_config
)
tokenizer = AutoTokenizer.from_pretrained(READER_MODEL_NAME)

READER_LLM = pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    do_sample=True,
    temperature=0.2,
    repetition_penalty=1.1,
    return_full_text=False,
    max_new_tokens=1024,
)

`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

In [ ]:
from transformers import Pipeline
import shelve

def get_chat_history(chat_id):
  """
  Retrieves the last N questions and answers from the chat history database.

  Args:
      chat_id (int): The unique identifier of the chat.

  Returns:
      str: A formatted string containing the last N questions and answers.
  """

  with shelve.open('chat_history') as db:
    try:
      history = db[str(chat_id)]
      formatted_history = "\n".join(history[-4:])  # Get the last 4 entries
    except KeyError:
      formatted_history = "No chat history available yet."

  return formatted_history

def update_chat_history(chat_id, question):
  """
  Updates the chat history database with the latest question.

  Maintains a maximum of N entries.

  Args:
      chat_id (int): The unique identifier of the chat.
      question (str): The user's latest question.
  """

  with shelve.open('chat_history') as db:
    try:
      history = db.get(str(chat_id), [])
      history.append(question)
      history = history[-4:]  # Keep only the last 4 entries
      db[str(chat_id)] = history
    except KeyError:
      db[str(chat_id)] = [question]

prompt_in_chat_format = [
  {
    "role": "system",
    "content": """Considering ONLY the following user question, provide a comprehensive answer.
    Respond only to the question asked, response should be concise and relevant."""
  },
  {
    "role": "user",
    "content": """Context:
    {context}
    ---
    Now here is the question you need to answer.

    Question: {question}"""
  },
]
RAG_PROMPT_TEMPLATE = tokenizer.apply_chat_template(
    prompt_in_chat_format, tokenize=False, add_generation_prompt=True
)
print(RAG_PROMPT_TEMPLATE)


<|system|>
Considering ONLY the following user question, provide a comprehensive answer.
    Respond only to the question asked, response should be concise and relevant.</s>
<|user|>
Context:
    {context}
    ---
    Now here is the question you need to answer.

    Question: {question}</s>
<|assistant|>



In [ ]:
!pip install ragatouille

  Using cached langchain-0.1.20-py3-none-any.whl (1.0 MB)
  Using cached langchain_core-0.1.52-py3-none-any.whl (302 kB)
  Using cached langchain_community-0.0.38-py3-none-any.whl (2.0 MB)
  Using cached langchain_text_splitters-0.0.2-py3-none-any.whl (23 kB)
  Attempting uninstall: langchain_core
    Found existing installation: langchain-core 0.2.1
    Uninstalling langchain-core-0.2.1:
      Successfully uninstalled langchain-core-0.2.1
  Attempting uninstall: langchain-text-splitters
    Found existing installation: langchain-text-splitters 0.2.0
    Uninstalling langchain-text-splitters-0.2.0:
      Successfully uninstalled langchain-text-splitters-0.2.0
  Attempting uninstall: langchain-community
    Found existing installation: langchain-community 0.2.1
    Uninstalling langchain-community-0.2.1:
      Successfully uninstalled langchain-community-0.2.1
  Attempting uninstall: langchain
    Found existing installation: langchain 0.2.1
    Uninstalling langchain-0.2.1:
      Succe

In [ ]:
from ragatouille import RAGPretrainedModel

RERANKER = RAGPretrainedModel.from_pretrained("colbert-ir/colbertv2.0")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [ ]:
from typing import Optional, List, Tuple
from langchain.docstore.document import Document as LangchainDocument


In [ ]:
from transformers import Pipeline


def answer_with_rag(
    question: str,
    llm: Pipeline,
    knowledge_index: FAISS,
    reranker: Optional[RAGPretrainedModel] = None,
    num_retrieved_docs: int = 30,
    num_docs_final: int = 5,
) -> Tuple[str, List[LangchainDocument]]:
    # Gather documents with retriever
    print("=> Retrieving documents...")
    relevant_docs = knowledge_index.similarity_search(
        query=question, k=num_retrieved_docs
    )
    relevant_docs = [doc.page_content for doc in relevant_docs]  # Keep only the text

    # Optionally rerank results
    if reranker:
        print("=> Reranking documents...")
        relevant_docs = reranker.rerank(question, relevant_docs, k=num_docs_final)
        relevant_docs = [doc["content"] for doc in relevant_docs]

    relevant_docs = relevant_docs[:num_docs_final]

    # Build the final prompt
    context = "\nExtracted documents:\n"
    context += "".join(
        [f"Document {str(i)}:::\n" + doc for i, doc in enumerate(relevant_docs)]
    )

    final_prompt = RAG_PROMPT_TEMPLATE.format(question=question, context=context)

    # Redact an answer
    print("=> Generating answer...")
    answer = llm(final_prompt)[0]["generated_text"]

    return answer, relevant_docs

In [ ]:
question = "What is NLP types?"



answer, relevant_docs = answer_with_rag(
   question, READER_LLM, VECTORDB, reranker=RERANKER
)

=> Retrieving documents...
=> Reranking documents...


100%|██████████| 1/1 [00:00<00:00,  4.84it/s]


=> Generating answer...


In [ ]:
print("==================================Answer==================================")
print(f"{answer}")
print("==================================Source docs==================================")
for i, doc in enumerate(relevant_docs):
    print(f"Document {i}------------------------------------------------------------")
    print(doc)

==================================Answer==================================
Answer: NLP types refer to the different analysis methods used in natural language processing. These include lexical analysis, syntactic analysis, semantic analysis, discourse integration, and pragmatic analysis. Each type involves specific techniques and goals to identify and analyze the structure of words, arrange them for grammar, determine their exact meaning, ensure meaningfulness, consider relationships between sentences, and reinterpret language based on real-world knowledge.
==================================Source docs==================================
Document 0------------------------------------------------------------
NLP Analysis Types-Lexical Analysis: Involves identifying and analyzing the structure of words, dividing text into paragraphs, sentences, and words. Lexicon represents the collection of words and phrases in a language.-Syntactic Analysis: Analyzes words in a sentence for grammar and ar

In [ ]:
question = "how to remove tashkeel from arabic?"

answer, relevant_docs = answer_with_rag(
    question, READER_LLM, VECTORDB, reranker=RERANKER
)

=> Retrieving documents...


KeyboardInterrupt: 

In [ ]:
print("==================================Answer==================================")
print(f"{answer}")
print("==================================Source docs==================================")
for i, doc in enumerate(relevant_docs):
    print(f"Document {i}------------------------------------------------------------")
    print(doc)

==================================Answer==================================
To remove tashkeel (diacritical marks) from Arabic text, you can use the `strip_tashkeel()` function provided by the Python library `pyarabic.araby`. Here's an example:

```python
from pyarabic.araby import strip_tashkeel

text = "هَلْ ذَهَبْتَ إِلَى المَكْتَبَةِ؟"
normalized_text = strip_tashkeel(text)
print(normalized_text)
```

This will output:

```
hL zhBt 'l mkTbḥ?
```

Alternatively, you can also use the `dediac_ar()` function from the CAMeL Tools library, which performs both dediacritization and orthographic normalization:

```python
from camel_tools.utils.dediac import dediac_ar

sentence = "هَلْ ذَهَبْتَ إِلَى المَكْتَبَةِ؟"
sentence_dediac = dediac_ar(sentence)
print(sentence_dediac)
```

This will output:

```
hL zhBt il mkTbḤ?
```

In general, it's recommended to perform dediacritization before any other preprocessing steps, as mentioned in Document 3.
==================================Source docs==

In [ ]:
!pip install python-telegram-bot==13.7

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 490.1/490.1 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.9/58.9 kB 10.0 MB/s eta 0:00:00
  Attempting uninstall: cachetools
    Found existing installation: cachetools 5.3.3
    Uninstalling cachetools-5.3.3:
      Successfully uninstalled cachetools-5.3.3


In [ ]:
from telegram.ext import Updater, MessageHandler, Filters,CommandHandler
from transformers import Pipeline

# Initialize list to store last four questions
chat_history = []

def handle_message(update, context):
    global chat_history
    question = update.message.text

    # Add the current question to the chat history
    chat_history.append(question)

    # Keep only the last four questions
    chat_history = chat_history[-4:]

    # Build prompt using the last four questions
    prompt = "\n".join(chat_history)

    # Get answer using chat history as context (replace with your implementation)
    answer, relevant_docs = answer_with_rag(prompt, READER_LLM, VECTORDB, reranker=RERANKER)
    update.message.reply_text(answer)
    follow_up_question = "Does your next question depend on my previous answer? (Click /YES or /NO).If you are finished, click /Exit."
    update.message.reply_text(follow_up_question)

def handle_exit(update, context):
    """
    Handles the /exit command, sends a thank you message, and stops the bot.
    """
    update.message.reply_text("Thank you for using the bot!.")
    context.bot.stop()  # Stops the bot from polling for new messages

def start(update, context):
  context.bot.send_message(chat_id=update.message.chat_id, text="Hi, I am your teach assistant I am here for any questions")



def yes(update, context):
  context.bot.send_message(chat_id=update.message.chat_id, text="Enter your next question")

def no(update, context):
  context.bot.send_message(chat_id=update.message.chat_id, text="Enter your next question")
  chat_history.clear()
  print(chat_history)


def main():
    updater = Updater('YOUR_TOKEN', use_context=True)
    dispatcher = updater.dispatcher

    text_question_handler = MessageHandler(Filters.text & ~Filters.command, handle_message)
    start_handler = CommandHandler('start', start)
    yes_handler = CommandHandler('YES', yes)
    no_handler = CommandHandler('NO', no)
    exit_handler = CommandHandler('Exit', handle_exit)

    dispatcher.add_handler(start_handler)
    dispatcher.add_handler(text_question_handler)
    dispatcher.add_handler(yes_handler)
    dispatcher.add_handler(no_handler)
    dispatcher.add_handler(exit_handler)

    updater.start_polling()
    updater.idle()


if __name__ == '__main__':
    main()


=> Retrieving documents...
=> Reranking documents...


100%|██████████| 1/1 [00:00<00:00, 10.31it/s]

=> Generating answer...



ERROR:telegram.ext.dispatcher:No error handlers are registered, logging exception.
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/telegram/ext/dispatcher.py", line 555, in process_update
  File "/usr/local/lib/python3.10/dist-packages/telegram/ext/handler.py", line 198, in handle_update
  File "<ipython-input-55-41f0accb0171>", line 31, in handle_exit
    context.bot.stop()  # Stops the bot from polling for new messages
AttributeError: 'ExtBot' object has no attribute 'stop'
